------------------------------------------------------------------------

# WES 207 SBUS to PID Demo Code

### Author: Aaron Coffman

-------------------------------------------------------------------------

### Jupyter Notebook Setup for Overlay Import

In [ ]:

from pynq import Overlay
from pynq import MMIO
import time
import numpy as np
from math import *

print('')
print('done')

### Loading FPGA bit stream

In [ ]:
# Load bitstream
#ol = Overlay('SBUS_DEMO_100M_tx_ext_clks_test5.bit')
ol = Overlay('FLIGHT_MAIN_RC_PID_DEMO.bit')

# Download bitstream to FPGA
t_before_bitstream = time.time()
ol.download()
t_after_bitstream = time.time()
print('  ')
print(t_after_bitstream - t_before_bitstream, 'seconds to program bitstream')

print('')
print('done')

### Setting MMIO Address Definitions

In [ ]:
# Address space definitions

## MMIO targets
UART_16550_BASE_ADDRESS         = 0x43C00000
UART_DRIVER_BASE_ADDRESS        = 0x43C10000
#UART_DATA_BASE_ADDRESS         = 0x40004000
RC_RECEIVER_BASE_ADDRESS        = 0x40000000
RC_RECEIVER_DATA_BASE_ADDRESS   = 0x40000020  # 0x20 for raw SBUS data
RC_RECEIVER_TEST_BASE_ADDRESS   = 0x40004000
FLIGHT_MAIN_CTRL_BASE_ADDRESS   = 0x40010000
FLIGHT_MAIN_CMD_BASE_ADDRESS    = 0x40020010  # 0x10 for scaled SBUS data
FLIGHT_MAIN_TEST_BASE_ADDRESS   = 0x40064000
PID_CTRL_BASE_ADDRESS           = 0x40030000
PID_KP_BASE_ADDRESS             = 0x40030020
PID_KI_BASE_ADDRESS             = 0x40030050
PID_KD_BASE_ADDRESS             = 0x40030040
PID_INPUT_BASE_ADDRESS          = 0x40040010  # 0x10 CMD, 0x20 meas
PID_TEST_BASE_ADDRESS           = 0x40054000

# MMIO address ranges
ADDRESS_RANGE1 = 0x1000   # 4k
ADDRESS_RANGE2 = 0x4000   # 16k
ADDRESS_RANGE3 = 0x10000  # 64k

print('Address Definitions complete')
print('')
print('done')

### Defining MMIO index offsets for read/write functions

In [ ]:
# defining MMIO index offsets
index_0 = 0x00
index_1 = 0x04
index_2 = 0x08
index_3 = 0x0C
index_4 = 0x10
index_5 = 0x14
index_6 = 0x18
index_7 = 0x1C 
index_8 = 0x20
index_9 = 0x24
index_10 = 0x28
index_11 = 0x2C
index_12 = 0x30
index_13 = 0x34
index_14 = 0x38
index_15 = 0x3C 
index_16 = 0x40
index_17 = 0x44
index_18 = 0x48
index_19 = 0x4C
index_20 = 0x50
index_21 = 0x54
index_22 = 0x58
index_23 = 0x5C 
index_24 = 0x60
index_25 = 0x64
index_26 = 0x68
index_27 = 0x6C
index_28 = 0x70
index_29 = 0x74

print('')
print('done')

### Configuring PID Controller Gains

In [ ]:
# calculating PID gains for 32 bit fixed point kp, ki, kd

Desired_gain = 0.200
val = Desired_gain / pow(2,-13)

print('Decimal Value: ', val   )
print('Hex Value: \t{0:x}'.format( int(val) ))


In [ ]:

# creating MMIO objects
kp_gains = MMIO(PID_KP_BASE_ADDRESS,ADDRESS_RANGE1, False)
ki_gains = MMIO(PID_KI_BASE_ADDRESS,ADDRESS_RANGE1, False)
kd_gains = MMIO(PID_KD_BASE_ADDRESS,ADDRESS_RANGE1, False)

# setting PID gains for DEMO/debug
kp_gains.write(index_0,0x666)   # Positon Roll  kp
kp_gains.write(index_1,0x666)   # Positon Pitch kp
kp_gains.write(index_2,0x666)   # Positon Yaw   kp
kp_gains.write(index_3,0x666)   # Rate    Roll  kp
kp_gains.write(index_4,0x666)   # Rate    Pitch kp
kp_gains.write(index_5,0x666)   # Rate    yaw   kp

ki_gains.write(index_0,0x0)   # Positon Roll   ki
ki_gains.write(index_1,0x0)   # Positon Pitch  ki
ki_gains.write(index_2,0x0)   # Rate    Roll   ki
ki_gains.write(index_3,0x0)   # Rate    Pitch  ki

kd_gains.write(index_0,0x4)   # Positon Roll   kd
kd_gains.write(index_1,0x4)   # Positon Pitch  kd
kd_gains.write(index_2,0x4)   # Rate    Roll   kd
kd_gains.write(index_3,0x4)   # Rate    Pitch  kd

# reading back PID gain values
# kp
test1 = kp_gains.read(index_0) * pow(2,-13)
test2 = kp_gains.read(index_1) * pow(2,-13)
test3 = kp_gains.read(index_2) * pow(2,-13)
test4 = kp_gains.read(index_3) * pow(2,-13)
test5 = kp_gains.read(index_4) * pow(2,-13)
test6 = kp_gains.read(index_5) * pow(2,-13)
print("kp:", test1, test2, test3, test4, test5, test6)

# ki
test1 = ki_gains.read(index_0) * pow(2,-13)
test2 = ki_gains.read(index_1) * pow(2,-13)
test3 = ki_gains.read(index_2) * pow(2,-13)
test4 = ki_gains.read(index_3) * pow(2,-13)
print("ki:", test1, test2, test3, test4)

# kd
test1 = kd_gains.read(index_0) * pow(2,-13)
test2 = kd_gains.read(index_1) * pow(2,-13)
test3 = kd_gains.read(index_2) * pow(2,-13)
test4 = kd_gains.read(index_3) * pow(2,-13)
print("kd:", test1, test2, test3, test4)


print('')
print('done')

### Creating MMIO Objects

In [ ]:

# UART16650 Core
UART_core = MMIO(UART_16550_BASE_ADDRESS,ADDRESS_RANGE3, False)   # 64k
UART_Driver = MMIO(UART_DRIVER_BASE_ADDRESS,ADDRESS_RANGE1, False)

# RC Receiver Core
RC_Driver = MMIO(RC_RECEIVER_BASE_ADDRESS,ADDRESS_RANGE1, False)
RC_Input_Data = MMIO(RC_RECEIVER_DATA_BASE_ADDRESS,ADDRESS_RANGE1, False)
RC_TEST_Data = MMIO(RC_RECEIVER_TEST_BASE_ADDRESS,ADDRESS_RANGE2, False)

# Flight Main Core
Flight_Main_core = MMIO(FLIGHT_MAIN_CTRL_BASE_ADDRESS,ADDRESS_RANGE1, False)
Flight_Main_Input_Data = MMIO(FLIGHT_MAIN_CMD_BASE_ADDRESS,ADDRESS_RANGE1, False)
Flight_Main_TEST_Data = MMIO(FLIGHT_MAIN_TEST_BASE_ADDRESS,ADDRESS_RANGE2, False)

# PID Core
PID_core = MMIO(PID_CTRL_BASE_ADDRESS,ADDRESS_RANGE1, False)
PID_Input_Data = MMIO(PID_INPUT_BASE_ADDRESS,ADDRESS_RANGE1, False)
PID_Test_Data = MMIO(PID_TEST_BASE_ADDRESS,ADDRESS_RANGE2, False)   # 16k

# UART scratch register address
SCRATCH_REG      =   0x101C 

# setting WHO_AM_I value for debug
#UART_core.write(SCRATCH_REG,0xFF)

print('UART16550 Core Configuration Complete')
print('')
print('done')

### Starting HLS Cores

In [ ]:
# NOTE, starting last core in chain first
# PID -> FLight_Main -> RC_Receiver -> UART_Driver

# setting ap_start/auto reset on PID core
#PID_core.write(0x00,0x01)  # runs once
PID_core.write(0x00,0x81) # runs continuously

# setting ap_start/auto reset on Flight main
#Flight_Main_core.write(0x00,0x01)  # runs once
Flight_Main_core.write(0x00,0x81) # runs continuously

# setting ap_start/auto reset on RC Receiver
#RC_Driver.write(0x00,0x01)  # runs once
RC_Driver.write(0x00,0x81) # runs continuously

# setting ap_start/auto reset on UART Driver
#UART_Driver.write(0x00,0x01)  # runs once
UART_Driver.write(0x00,0x81) # runs continuously


# reading back CONFIG registers for HLS cores
CONFIG_REG4 = PID_core.read(0x00)
CONFIG_REG3 = Flight_Main_core.read(0x00)
CONFIG_REG2 = RC_Driver.read(0x00)
CONFIG_REG1 = UART_Driver.read(0x00)


print('CONFIG_REG UART: \t', CONFIG_REG1)
print('CONFIG_REG RC: \t\t', CONFIG_REG2)
print('CONFIG_REG Flight Main: ', CONFIG_REG3)
print('CONFIG_REG PID: \t', CONFIG_REG4)
print('')
print('')

### UART Driver Configuration Confirmation and Status

In [ ]:

# UART16650 Line status register offset
LINE_STATUS_REG  =   0x1014 

# reading scratch register for configuration verification, expect 0x45
test_read = UART_core.read(SCRATCH_REG)
print('WHO_AM_I: \t' + str(test_read))

# reading FIFO status of UART core
# Overrun Error = 1 indicates RX FIFO Overflow
OVERRUN_ERROR = UART_core.read(LINE_STATUS_REG)
OVERRUN_ERROR = OVERRUN_ERROR & 0x02 # grabbing bit 1
print('OVERRUN_ERROR: ', OVERRUN_ERROR,'\r')


print('')
print('done')

### RC Receiver MMIO reads for data flow verification

In [ ]:

# reading RC Receiver input data
test1 = RC_TEST_Data.read(index_0) 
test2 = RC_TEST_Data.read(index_1)
test3 = RC_TEST_Data.read(index_2)
test4 = RC_TEST_Data.read(index_3)
test5 = RC_TEST_Data.read(index_4)
test6 = RC_TEST_Data.read(index_5)

# reading RC Receiver output data
test7 = RC_TEST_Data.read(index_6)
test8 = RC_TEST_Data.read(index_7)
if( test8 > 0x7FFFFFFF ):    # 2147483647
    test8 = -(0xFFFFFFFF - test8 + 1)  # 4294967295
test9 = RC_TEST_Data.read(index_8)
if( test9 > 0x7FFFFFFF ):
    test9 = -(0xFFFFFFFF - test9 + 1)
test10 = RC_TEST_Data.read(index_9)
if( test10 > 0x7FFFFFFF ):
    test10 = -(0xFFFFFFFF - test10 + 1)
test11 = RC_TEST_Data.read(index_10)
test12 = RC_TEST_Data.read(index_11)

print('Throttle Input RC Core: \t', "%4d"% (test1 >> 13))
print('Roll Input RC Core: \t\t', "%4d"% (test2 >> 13))
print('Pitch Input RC Core: \t\t', "%4d"% (test3 >> 13))
print('Yaw Input RC Core: \t\t', "%4d"% (test4 >> 13))
print('Arm Input RC Core: \t\t', "%1d"% (test5 >> 13))
print('Mode Input RC Core: \t\t', "%1d"% (test6 >> 13))
print('')
print('')
print('Throttle Output RC Core: \t', "%1.5f"%((test7 * pow(2,-13))))
print('Roll Output RC Core: \t\t', "%1.5f"%((test8 * pow(2,-13))))
print('Pitch Output RC Core: \t\t', "%1.5f"%(test9 * pow(2,-13)))
print('Yaw Output RC Core: \t\t', "%1.5f"%(test10 * pow(2,-13)))
print('Arm Output RC Core: \t\t', "%1d"% (test11 * pow(2,-13)))
print('Mode Output RC Core: \t\t', "%1d"% (test12 * pow(2,-13)))

print('')
print('done')

### Flight Main MMIO reads for data flow verification

In [ ]:

# reading Flight Main input data
test1 = Flight_Main_TEST_Data.read(index_0)
if( test1 > 0x7FFFFFFF ):    # 2147483647
    test1 = -(0xFFFFFFFF - test1 + 1)  # 4294967295
test2 = Flight_Main_TEST_Data.read(index_1)
if( test2 > 0x7FFFFFFF ):    # 2147483647
    test2 = -(0xFFFFFFFF - test2 + 1)  # 4294967295
test3 = Flight_Main_TEST_Data.read(index_2)
if( test3 > 0x7FFFFFFF ):    # 2147483647
    test3 = -(0xFFFFFFFF - test3 + 1)  # 4294967295
test4 = Flight_Main_TEST_Data.read(index_3)
if( test4 > 0x7FFFFFFF ):    # 2147483647
    test4 = -(0xFFFFFFFF - test4 + 1)  # 4294967295
test5 = Flight_Main_TEST_Data.read(index_4)
test6 = Flight_Main_TEST_Data.read(index_5)

# reading Flight Main output data
test7 = Flight_Main_TEST_Data.read(index_6)
if( test7 > 0x7FFFFFFF ):    # 2147483647
    test7 = -(0xFFFFFFFF - test7 + 1)  # 4294967295
test8 = Flight_Main_TEST_Data.read(index_7)
if( test8 > 0x7FFFFFFF ):    # 2147483647
    test8 = -(0xFFFFFFFF - test8 + 1)  # 4294967295
test9 = Flight_Main_TEST_Data.read(index_8)
if( test9 > 0x7FFFFFFF ):    # 2147483647
    test9 = -(0xFFFFFFFF - test9 + 1)  # 4294967295
test10 = Flight_Main_TEST_Data.read(index_9)
if( test10 > 0x7FFFFFFF ):    # 2147483647
    test10 = -(0xFFFFFFFF - test10 + 1)  # 4294967295
test11 = Flight_Main_TEST_Data.read(index_10)
test12 = Flight_Main_TEST_Data.read(index_11)

print('Throttle Input Flight Main: \t',"%1.5F"% (test1 * pow(2,-13)))
print('Roll Input Flight Main: \t',"%1.5F"% (test2 * pow(2,-13)))
print('Pitch Input Flight Main: \t',"%1.5F"% (test3 * pow(2,-13)))
print('Yaw Input Flight Main: \t\t',"%1.5F"% (test4 * pow(2,-13)))
print('Arm Input Flight Main: \t\t',"%1d"% (test5 * pow(2,-13)))
print('Mode Input Flight Main: \t',"%1d"% (test6 * pow(2,-13)))
print('')
print('')
print('Throttle Output Flight Main: \t',"%1.5F"% (test7 * pow(2,-13)))
print('Roll Output Flight Main: \t',"%1.5F"% (test8 * pow(2,-13)))
print('Pitch Output Flight Main: \t',"%1.5F"% (test9 * pow(2,-13)))
print('Yaw Output Flight Main: \t',"%1.5F"% (test10 * pow(2,-13)))
print('Arm Output Flight Main: \t',"%1d"% (test11 * pow(2,-13)))
print('Mode Output Flight Main: \t',"%1d"% (test12 * pow(2,-13)))




print('')
print('done')

### PID MMIO reads for data flow verification

In [ ]:

# reading PID input data
TEST_READ1  = PID_Test_Data.read(index_0)
if( TEST_READ1 > 0x7FFFFFFF ):    # 2147483647
    TEST_READ1 = -(0xFFFFFFFF - TEST_READ1 + 1)  # 4294967295
TEST_READ2  = PID_Test_Data.read(index_1)
if( TEST_READ2 > 0x7FFFFFFF ):    # 2147483647
    TEST_READ2 = -(0xFFFFFFFF - TEST_READ2 + 1)  # 4294967295
TEST_READ3  = PID_Test_Data.read(index_2)
if( TEST_READ3 > 0x7FFFFFFF ):    # 2147483647
    TEST_READ3 = -(0xFFFFFFFF - TEST_READ3 + 1)  # 4294967295
TEST_READ4  = PID_Test_Data.read(index_3)
if( TEST_READ4 > 0x7FFFFFFF ):    # 2147483647
    TEST_READ4 = -(0xFFFFFFFF - TEST_READ4 + 1)  # 4294967295
TEST_READ5  = PID_Test_Data.read(index_4)
TEST_READ6  = PID_Test_Data.read(index_5)

# reading PID motor outputs
TEST_READ7  = PID_Test_Data.read(index_6)
TEST_READ8  = PID_Test_Data.read(index_7)
TEST_READ9  = PID_Test_Data.read(index_8)
TEST_READ10 = PID_Test_Data.read(index_9)
TEST_READ11 = PID_Test_Data.read(index_10)
TEST_READ12 = PID_Test_Data.read(index_11)
TEST_READ13 = PID_Test_Data.read(index_12)
TEST_READ14 = PID_Test_Data.read(index_13)


print('Throttle Input PID: \t', "%1.5F"% (TEST_READ1 * pow(2,-13)))
print('Roll Input PID: \t', "%1.5F"% (TEST_READ2 * pow(2,-13)))
print('Pitch Input PID: \t', "%1.5F"% (TEST_READ3 * pow(2,-13)))
print('Yaw Input PID: \t\t', "%1.5F"% (TEST_READ4 * pow(2,-13)))
print('ARM Flag PID: \t\t', "%1d"% (TEST_READ5 * pow(2,-13)))
print('Flight Mode Flag PID: \t', "%1d"% (TEST_READ6 * pow(2,-13)))
print('')
print('')
print('M1 out PID: \t', "%1.3F"% (TEST_READ7 * pow(2,-13)))
print('M2 out PID: \t', "%1.3F"% (TEST_READ8 * pow(2,-13)))
print('M3 out PID: \t', "%1.3F"% (TEST_READ9 * pow(2,-13)))
print('M4 out PID: \t', "%1.3F"% (TEST_READ10 * pow(2,-13)))
print('M5 out PID: \t', "%1.3F"% (TEST_READ11 * pow(2,-13)))
print('M6 out PID: \t', "%1.3F"% (TEST_READ12 * pow(2,-13)))
print('M7 out PID: \t', "%1.3F"% (TEST_READ13 * pow(2,-13)))
print('M8 out PID: \t', "%1.3F"% (TEST_READ14 * pow(2,-13)))


print('')
print('done')

# RC Receiver Data Flow to PID controller Real-Time Demos

### RC Inputs to PID Demo

In [ ]:
# RC Receiver Demo - RC controls
from __future__ import print_function
import sys
count = 0

print('')
print('')
while(count < 10000):
    
    # readign PID input data - RC control
    TEST_READ1  = PID_Test_Data.read(index_0)
    if( TEST_READ1 > 0x7FFFFFFF ):    # 2147483647
        TEST_READ1 = -(0xFFFFFFFF - TEST_READ1 + 1)  # 4294967295
    TEST_READ1 = TEST_READ1 * pow(2,-13)
    TEST_READ2  = PID_Test_Data.read(index_1)
    if( TEST_READ2 > 0x7FFFFFFF ):    # 2147483647
        TEST_READ2 = -(0xFFFFFFFF - TEST_READ2 + 1)  # 4294967295
    TEST_READ2 = TEST_READ2 * pow(2,-13)
    TEST_READ3  = PID_Test_Data.read(index_2)
    if( TEST_READ3 > 0x7FFFFFFF ):    # 2147483647
        TEST_READ3 = -(0xFFFFFFFF - TEST_READ3 + 1)  # 4294967295
    TEST_READ3 = TEST_READ3 * pow(2,-13)
    TEST_READ4  = PID_Test_Data.read(index_3)
    if( TEST_READ4 > 0x7FFFFFFF ):    # 2147483647
        TEST_READ4 = -(0xFFFFFFFF - TEST_READ4 + 1)  # 4294967295
    TEST_READ4 = TEST_READ4 * pow(2,-13)
    TEST_READ5  = PID_Test_Data.read(index_4) * pow(2,-13)
    TEST_READ6  = PID_Test_Data.read(index_5) * pow(2,-13)
    
    
    #print('')
    print('T: ', "%1.3F"% (TEST_READ1),'  R: ',"%1.3F"% (TEST_READ2),'  P: ',"%1.3F"% (TEST_READ3),'  Y: ',"%1.3F"% (TEST_READ4),'  A: ',"%1d"% (TEST_READ5),'  M: ',"%1d"% (TEST_READ6),'  ', end='\r')
    
    time.sleep(0.150) 
    sys.stdout.flush()
    count = count + 1
print("")

#### RC Commands Scaled

### PID Outputs to PWM Demo

In [ ]:
# RC Receiver Demo  Motor Outputs
from __future__ import print_function
import sys
count = 0

print('')
print('')
while(count < 10000):
    
    #motor outputs
    TEST_READ7  = PID_Test_Data.read(index_6) * pow(2,-13)
    TEST_READ8  = PID_Test_Data.read(index_7) * pow(2,-13)
    TEST_READ9  = PID_Test_Data.read(index_8) * pow(2,-13)
    TEST_READ10 = PID_Test_Data.read(index_9) * pow(2,-13)
    TEST_READ11 = PID_Test_Data.read(index_10) * pow(2,-13)
    TEST_READ12 = PID_Test_Data.read(index_11) * pow(2,-13)
    TEST_READ13 = PID_Test_Data.read(index_12) * pow(2,-13)
    TEST_READ14 = PID_Test_Data.read(index_13) * pow(2,-13)

    #print('')
    print('M1: ', "%1.3F"% (TEST_READ7),' M2: ',"%1.3F"% (TEST_READ8),' M3: ',"%1.3F"% (TEST_READ9),' M4: ',"%1.3F"% (TEST_READ10),' M5: ',"%1.3F"% (TEST_READ11),' M6: ',"%1.3F"% (TEST_READ12),' M7: ',"%1.3F"% (TEST_READ13),' M8: ',"%1.3F"% (TEST_READ14),'  ', end='\r')
    
    time.sleep(0.150) 
    sys.stdout.flush()
    count = count + 1
print("")

#### PID Motor Commands

### End of DEMO code